# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

In [ ]:
# With set(date_sorter().index) ^ set([x for x in range(0, 500)]) I found the missing lines
# Missing lines ->
# 25: 4-13-82 Other Child Mental Health Outcomes Scales Used
# 39: 1-14-81 Communication with referring physician?: Done
# 40: 7-29-75 CPT Code: 90801 - Psychiatric Diagnosis Interview
# 71: 4-13-89 Communication with referring physician?: Not Done

# Rows duplicating ->
# 11: (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent
# 73: Lithium 0.25 (7/11/77).  LFTS wnl.  Urine tox neg.  Serum tox + fluoxetine 500; otherwise neg.  TSH 3.28.  BUN/Cr: 16/0.83.  Lipids unremarkable.  B12 363, Folate >20.  CBC: 4.9/36/308 Pertinent Medical Review of Systems Constitutional:

In [186]:
def date_sorter():
    import re
    import pandas as pd

    doc = []
    with open('assets/dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    

    date_pattern = """
    (
    
    # date in format: <04/20/2009, 04/20/09, 4/20/09, 4/3/09>
    (?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4})
    |
    
    # date in format <Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009>
    (?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.*[-\s]?\d{1,2},*\s\d{2,4})
    |
    
    # date in format <20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009>
    (?:\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.*,*\s\d{2,4})
    |
    
    # date in format <Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009>
    (?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{1,2}(?:th|st|nd),*\s\d{2,4})
    |
    
    # date in format <Feb 2009; Sep 2009; Oct 2010>
    (?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,*\s\d{2,4})
    |
    
    # date in format <6/2008; 12/2009>
    (?:\d{1,2}/\d{2,4})
    |
    
    # date in format 2009; 2010
    (?:(?:(?:19|20)\d{2})|(?:\b\d{2}\b))
    )
    """
    
    df = df.str.strip()
    df = df.str.extractall(date_pattern, flags=re.VERBOSE)
    
    # Group by index level 0 and keep first occurrence
    df = df.groupby(level=0).first()
    df = df.rename(columns = {0: 'text'})
    
    # Noticed that indexes 298 and 313 has date mispeling. I will fix it manually.
    df.at[298, 'text'] = 'January 1993'
    df.at[313, 'text'] = 'December 1978'
    
    # Adding '19' to date's year in format <xx/xx/yy>. Ex.: 05/12/56 -> 05/12/1956 
    df.loc[df['text'].str.contains(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2}$'), 'text'] = df.loc[df['text'].str.contains(r'^\d{1,2}[/-]\d{1,2}[/-]\d{2}$'), 'text'].map(lambda x: x[0:-2] + '19' + x[-2:])
    
    # Casting to Date
    df['DateTime'] = pd.to_datetime(df['text'])
    df = df.sort_index()
    df = df.sort_values('DateTime', ascending=True, kind='stable')
    
    order = pd.Series(df.index)
    return order